# Práctica 1.A Toma de contacto con AIMA

## Grupo 16:
- Daniela Alejandra Cordova Porta
- David Bugoi
- Erik Karlgren Domercq

La práctica está organizada en 3 partes. En la primera se muestra a través de ejemplos cómo se **representan** algunos problemas clásicos como el de las jarras o el problema del ocho puzzle. En la segunda parte se muestra el uso de los algoritmos de búsqueda. En la tercera parte aprenderemos a medir las propiedades de los algoritmos.
En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios.  

Cuando termines los ejercicios entrega este archivo en el campus. 

## Parte I: Representación de problemas de espacios de estados.

### El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. En esta parte en vez de importarla la hemos copiado aquí para la explicación.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros.

In [7]:
   class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [8]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [31]:
p =Jarras()
p.initial

(0, 0)

In [32]:
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [33]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [34]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [35]:
p.goal_test(p.initial)

False

### Problema de los misioneros

In [9]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        # cada accion tiene un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [37]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura la llamada sería:  breadth_first_tree_search(estado).solution()

In [38]:
misioneros.initial

(3, 3, 0)

In [39]:
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

### EJERCICIO 1. Completa la definición de los operadores en el problema del Puzle de 8. 

In [10]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        
        ### EJERCICIO 1.1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        
       ### EJERCICIO 1.2. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
            
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1

#### Una vez completada la definición de la clase podrás probar los siguientes ejemplos.

In [41]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [42]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [43]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [44]:
p8.result(p8.initial,"Mover hueco abajo")

IndexError: list index out of range

In [45]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [46]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

1

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

### Usaremos búsqueda en anchura y en profundidad para encontrar soluciones tanto al problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [17]:
cd aima-python

C:\Users\Daniela\Documents\UCM IA\Practica 1A\aima-python


In [18]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [35]:
## resolvemos el problema de las jarras con el método de búsqueda en anchura.  

In [36]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [54]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### Ejercicio 2. Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8

In [55]:
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura breadth_first_tree_search(misioneros).solution()

In [56]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [57]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [58]:
breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
# La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  
# Busqueda en anchura es completo.. ¿no debería terminar? escribe al final del ejercicio tus conclusiones.

KeyboardInterrupt: 

In [59]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [60]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [61]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['UP', 'LEFT', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'RIGHT', 'DOWN']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [62]:
depth_first_graph_search(estado).solution()

KeyboardInterrupt: 

In [95]:
breadth_first_graph_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

#### En este ejercicio se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones:

El algoritmo usando control de repetidos se ejecuta y termina ya que no entra en bucles infintos. De todos modos, no todos los estados iniciales en el puzle de 8 tienen solución, y por eso la búsqueda en anchura o en profundidad no termina a veces.
<p>
Podemos observar que la búsqueda en amplitud es mejor que la de profundidad en este problema, y eso es porque la búsqueda en profundidad sigue un solo camino para buscar una solución cuando el espacio de acciones posibles es inmenso y el problema se puede resolver en el mejor caso en unas pocas decenas de pasos como mucho. Es encontrar este mejor caso de bajo coste lo que se le da bien a la búsqueda en anchura.
<p>
También podemos destacar que el factor de ramificación y número de nodos expandidos de la busqueda en anchura es menor que el factor de ramificacion y profundidad máxima de la búsqueda en profundidad, así que a pesar de que los dos casos tienen complejidad exponencial en la búsqueda de la solución la búsqueda en anchura es mucho mejor en este caso.


### Ejercicio 3:  Definición de heurísticas

#### Para el problema de los misioneros define una heurística y comenta sus propiedades

Propiedades de `h2`:

- Esta heuristica relaja el problema pensando que los caníbales nunca se comen a los misioneros.
- Se considera que una persona conduce la barca de ida y vuelta y la otra es transportada (2*), por lo cual sólo una persona acaba cruzando a la otra orilla.
- `orilla` es 1 si está en el lado inicial y es 0 si está en el final (cómo está representado al revés, se hace un "not" del valor)
- `Cinicial` y `Minicial` correponde a los valores de los canibales y misioneros en el lado de la orilla inicial.
- De las heurísticas estudiadas en clase, ésta es la más informada (siendo admisible) y tiene un valor más cercano a h*, siendo así una heurística efectiva y coherente. 

In [2]:
# Heuristica para el problema de misioneros.

def h2(node):
    state = node.state
    Minicial = int(state[0])
    Cinicial = int(state[1])
    orilla   = int (state [2])
    if (orilla == 0):
        orilla=1
    else:
        orilla = 0
    resultado = 2*(Cinicial-Minicial) - orilla
    return resultado

In [101]:
breadth_first_graph_search(ProblemaMisioneros((3, 3, 0), (0, 0, 1))).solution()

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [100]:
best_first_graph_search(ProblemaMisioneros((3, 3, 0), (0, 0, 1)), h2).solution()

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1m', (1, 0)),
 ('1m1c', (1, 1))]

In [103]:
astar_search(ProblemaMisioneros((3, 3, 0), (0, 0, 1)), h2).solution()

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1m', (1, 0)),
 ('1m1c', (1, 1))]

#### Para el problema del puzle de 8 se pide definir al menos las siguientes funciones heurísticas:
* `linear(node)`: cuenta el número de casillas mal colocadas respecto al estado final.
* `manhattan(node)`: suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
* `max_heuristic(node)`: maximo de las dos anteriores
* `sqrt_manhattan(node)`:  raíz cuadrada de la distancia Manhattan

In [11]:
# Heuristicas para el 8 Puzzle. Puedes definir las funciones fuera de la clase ya que en la llamada a A* puedes pasar el nombre 
# de la función. 
import math

def linear(node):
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,0)
    state = node.state
    sum =0
    for i in range(len(state)):
        if(goal.index(i)!=state.index(i)):
            sum+=1
    return sum

def matrixLocation(x):
    if x==0:
        return 1,1
    elif x==1:
        return 1,2
    elif x==2:
        return 1,3
    elif x==3:
        return 2,1
    elif x==4:
        return 2,2
    elif x==5:
        return 2,3
    elif x==6:
        return 3,1
    elif x==7:
        return 3,2
    elif x==8:
        return 3,3

def manhattan(node):
    state = node.state
    goal = (1,2,3,4,5,6,7,8,0)
    mhd = 0
    for i in range(len(state)-1):
        x1,y1 = matrixLocation(goal.index(i))
        x2,y2 = matrixLocation(state.index(i))
        mhd = mhd + abs(x2-x1) + abs(y2-y1)
    
    return mhd

def sqrt_manhattan(node):
    state = node.state
    mhd = manhattan(node)
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)
        

### Ejercicio 4. 
Usar las implementaciones de los algoritmos que correspondan a búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema del 8 puzzle para el siguiente **estado inicial** y comparar los costes temporales usando %timeit y comentar los resultados.

              +---+---+---+
              | 2 | 4 | 3 |
              +---+---+---+
              | 1 | 5 | 6 |
              +---+---+---+
              | 7 | 8 | H |
              +---+---+---+


In [97]:
puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
astar_search(puzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [98]:
astar_search(puzle,manhattan).solution()

['Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [99]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [100]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [101]:
astar_search(puzle,linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [102]:
astar_search(puzle,max_heuristic).solution()

['Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco derecha']

¿Has notado diferencias en los tiempos de ejecución? Vamos a medirlo. Aunque las heurísticas no afectan a la solución obtenida sí hay diferencias importantes en el tiempo de cálculo.
<br>

In [103]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [195]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

3.23 ms ± 77 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [194]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

2.51 ms ± 197 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [106]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

28.8 ms ± 3.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [107]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

2.57 ms ± 14.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Escribe aquí tus conclusiones sobre qué heurística es mejor y por qué.

A pesar de que todas tienen la misma cantidad de movimientos, el tiempo con `manhattan` es menor que en el resto. Esto es entendible ya que es una heurística más informativa respecto a cuál es el mejor camino a tomar para resolver el puzle. De todos modos las búsquedas A* con heurísticas `linear`, `manhattan` y `max_heuristic` analizan una cantidad similar de nodos, por lo cual el tiempo de ejecución es semejante.

In [202]:
resuelve_ocho_puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0),astar_search,linear)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 8. Nodos analizados: 13


In [203]:
resuelve_ocho_puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0),astar_search,manhattan)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 8. Nodos analizados: 16


In [204]:
resuelve_ocho_puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0),astar_search,sqrt_manhattan)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: sqrt_manhattan
Longitud de la solución: 8. Nodos analizados: 41


In [205]:
resuelve_ocho_puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0),astar_search,max_heuristic)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: max_heuristic
Longitud de la solución: 8. Nodos analizados: 16


## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función `%timeit` para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil.. como hemos comentado al principio solo algunos tableros tienen solucion. En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando su paridad (o número de inversiones). Si es impar el tablero **no** tiene solución.  El concepto de inversión de una ficha será la suma del número de fichas que se encuentran en una posición superior a dicha ficha y que deberían estar situadas en una posición inferior. La inversión total será la suma de las inversiones individuales. Si este número es par, el puzzle tendrá solución. En caso contrario, no habrá solución. 

In [12]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. 
# The solvability of a configuration can be checked by calculating the Inversion Permutation. If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """

        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        
        return inversion % 2 == 0        

In [42]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

In [43]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [113]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [114]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1, manhattan).solution()

['Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [115]:
astar_search(p8, manhattan).solution()

['Mover hueco derecha']

In [116]:
astar_search(p8p, manhattan).solution()

['Mover hueco derecha']

In [13]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """

    p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
    if p8p.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [118]:
resuelve_ocho_puzzle(estado_inicial,astar_search,manhattan)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 1. Nodos analizados: 2


### Ejercicio 5:  resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

E1 = (2,1,3,4,8,6,7,0,5)
E2 = (1,0,3,4,8,6,7,2,5)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)

          E1              E2              E3              E4
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    
     | 2 | 1 | 3 |   | 1 | 0 | 3 |   | 4 | 5 | 6 |   | 1 | 2 | 3 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 4 | 8 | 6 |   | 4 | 8 | 6 |   | 1 | 0 | 3 |   | H | 5 | 6 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 7 | H | 5 |   | 7 | 2 | 5 |   | 7 | 8 | 2 |   | 4 | 7 | 8 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la
 siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo, y
 **justificar los resultados con las distintas propiedades teóricas estudiadas en clase**.  
 
 

-----------------------------------------------------------------------------------------
                                       E1             E2           E3           E4
    Anchura                            L=17           L=11         L=20         L=17
                                       T=6.53 s       T=62.9 ms    T=1min 48s   T=7.31 s  
                                       NA=13634       NA=1255      NA=57057     NA=13634 
   
    Profundidad                        L=7977         L=52429      L=39720      L=7977  
                                       T= 9.3 s       T=10.2 s     T=4min 35s   T=10.7 s 
                                       NA=8198        NA=60036     NA=42926     NA=8198 
                                       
    Coste Uniforme                     L=17           L=11         L=20         L=17
                                       T=45.4 s       T=199 ms     T=46.9 s     T=1min 3s  
                                       NA=14092       NA=870       NA=48428     NA=14092 
                                       
    Primero el mejor (linear)          L=39           L=33         L=76         L=39
                                       T=10 ms        T=2.69 ms    T=155 ms     T=10.6 ms  
                                       NA=173         NA=78        NA=748       NA=173
                                                                            
    Primero el mejor (manhattan)       L=69           L=35         L=70         L=69 
                                       T=72.7 ms      T=5.84 ms    T=36 ms      T=79.1 ms  
                                       NA=511         NA=98        NA=320       NA=511
                                                                             
    A* (linear)                        L=17           L=11         L=20         L=17 
                                       T=180 ms       T=2.95 ms    T=2.93 s     T=226 ms  
                                       NA=873         NA=77        NA=3377      NA=873
                                                                             
    A* (manhattan)                     L=17           L=11         L=20         L=17 
                                       T=44.1 ms      T=1.5 ms     T=442 ms     T=51 ms  
                                       NA=393         NA=42        NA=1288      NA=393

 -----------------------------------------------------------------------------------------




#### Justificación
Los algoritmos de búsqueda óptimos son claramente la _búsqueda en anchura,_ _la búsqueda con coste uniforme_ y _A*_ con las heurísticas `manhattan` y `linear`. Esto ocurre con la _búsqueda en anchura_ porque el operador de búsqueda tiene coste uniforme, y como además es siempre positivo también se garantiza que la _búsqueda con coste uniforme_ es óptima. En el caso de las búsquedas con _A*_ estas son óptimas porque `manhattan` y `linear` son heurísticas admisibles. En cambio, la _búsqueda en profundidad_ nunca garantiza encontrar la solución óptima, mientras que las búsquedas de _primero el mejor_ encuentran soluciones mucho más razonables (que no óptimas) debido al uso de heurísticas.

Si nos centramos, no obstante, en el tiempo de ejecución y el número de nodos expandidos observamos que la _búsqueda en anchura_ y _en profundidad_ son las menos eficientes ya que expanden nodos de forma arbitraria. La _búsqueda con coste uniforme_ a veces tarda incluso más porque el coste del camino de cada nodo coincide con su profundidad, así que se comporta por tanto como una _búsqueda por anchura_ con el coste añadido de una cola de prioridad. Por otro lado las búsquedas con heurísticas tardan mucho menos tiempo ya que eligen qué nodos expandir de forma informada y así acaban recorriendo menos nodos antes de encontrar la solución. De hecho, a veces las búsquedas de _primero el mejor_ a veces tardan menos en encontrar la solución que _A*_ porque no usan una cola de prioridad.

#### Definición de los estados iniciales y sus problemas correspondientes

##### E1

In [ ]:
estado_inicial = (2,1,3,4,8,6,7,0,5)
p8e1 = Ocho_Puzzle(estado_inicial)

##### E2

In [141]:
estado_inicial2 = (1,0,3,4,8,6,7,2,5)
p8e2 = Ocho_Puzzle(estado_inicial2)

##### E3

In [167]:
estado_inicial3 = (4,5,6,1,0,3,7,8,2)
p8e3 = Ocho_Puzzle(estado_inicial3)

##### E4

In [185]:
estado_inicial4 = (2,1,3,4,8,6,7,0,5)
p8e4 = Ocho_Puzzle(estado_inicial4)

#### Búsqueda en anchura

##### E1

In [120]:
resuelve_ocho_puzzle(estado_inicial, breadth_first_graph_search)

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 17. Nodos analizados: 13634


In [135]:
%%timeit
breadth_first_graph_search(p8e1).solution()

6.53 s ± 247 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### E2

In [143]:
resuelve_ocho_puzzle(estado_inicial2, breadth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 11. Nodos analizados: 1255


In [145]:
%%timeit
breadth_first_graph_search(p8e2).solution()

62.9 ms ± 5.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


##### E3

In [169]:
resuelve_ocho_puzzle(estado_inicial3, breadth_first_graph_search)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 20. Nodos analizados: 57057


In [170]:
%%timeit
breadth_first_graph_search(p8e3).solution()

1min 48s ± 3.94 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### E4

In [187]:
resuelve_ocho_puzzle(estado_inicial4, breadth_first_graph_search)

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 17. Nodos analizados: 13634


In [188]:
%%timeit
breadth_first_graph_search(p8e4).solution()

7.31 s ± 297 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#profundidad

### Búsqueda en profundidad

##### E1

In [122]:
resuelve_ocho_puzzle(estado_inicial,depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco der

In [136]:
%%timeit
depth_first_graph_search(p8e1).solution()

9.3 s ± 454 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### E2

In [211]:
resuelve_ocho_puzzle(estado_inicial2,depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derec

In [212]:
%%timeit
depth_first_graph_search(p8e2).solution()

10.2 s ± 278 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### E3

In [171]:
resuelve_ocho_puzzle(estado_inicial3,depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquie

In [172]:
%%timeit
depth_first_graph_search(p8e3).solution()

4min 35s ± 6.26 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### E4

In [189]:
resuelve_ocho_puzzle(estado_inicial4,depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco der

In [190]:
%%timeit
depth_first_graph_search(p8e4).solution()

10.7 s ± 792 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Búsqueda con coste uniforme

##### E1

In [124]:
resuelve_ocho_puzzle(estado_inicial,uniform_cost_search)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: uniform_cost_search
Longitud de la solución: 17. Nodos analizados: 14092


In [137]:
%%timeit
uniform_cost_search(p8e1).solution()

45.4 s ± 1.67 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### E2

In [146]:
resuelve_ocho_puzzle(estado_inicial2,uniform_cost_search)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: uniform_cost_search
Longitud de la solución: 11. Nodos analizados: 870


In [147]:
%%timeit
uniform_cost_search(p8e2).solution()

199 ms ± 12.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


##### E3

In [173]:
resuelve_ocho_puzzle(estado_inicial3,uniform_cost_search)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 20. Nodos analizados: 48428


In [213]:
%%timeit
uniform_cost_search(p8e3).solution()

46.9 s ± 1.5 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### E4

In [191]:
resuelve_ocho_puzzle(estado_inicial4,uniform_cost_search)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: uniform_cost_search
Longitud de la solución: 17. Nodos analizados: 14092


In [192]:
%%timeit
uniform_cost_search(p8e4).solution()

1min 3s ± 1.88 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Primero el mejor (linear)   

##### E1

In [125]:
resuelve_ocho_puzzle(estado_inicial, best_first_graph_search, linear)

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 39. Nodos analizados: 173

In [138]:
%%timeit
best_first_graph_search(p8e1, linear).solution()

10 ms ± 341 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### E2

In [152]:
resuelve_ocho_puzzle(estado_inicial2, best_first_graph_search, linear)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 33. Nodos analizados: 78


In [149]:
%%timeit
best_first_graph_search(p8e2, linear).solution()

2.69 ms ± 275 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### E3

In [175]:
resuelve_ocho_puzzle(estado_inicial3, best_first_graph_search, linear)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hue

In [176]:
%%timeit
best_first_graph_search(p8e3, linear).solution()

155 ms ± 9.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


##### E4

In [196]:
resuelve_ocho_puzzle(estado_inicial4, best_first_graph_search, linear)

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 39. Nodos analizados: 173

In [197]:
%%timeit
best_first_graph_search(p8e4, linear).solution()

10.6 ms ± 439 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Primero el mejor (manhattan)

##### E1

In [128]:
resuelve_ocho_puzzle(estado_inicial, best_first_graph_search, manhattan)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover

In [139]:
%%timeit
best_first_graph_search(p8e1,manhattan).solution()

72.7 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


##### E2

In [163]:
resuelve_ocho_puzzle(estado_inicial2, best_first_graph_search, manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: manhattan
Longitud de la solución: 35. Nodos analizados: 98


In [155]:
%%timeit
best_first_graph_search(p8e2,manhattan).solution()

5.84 ms ± 925 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### E3

In [177]:
resuelve_ocho_puzzle(estado_inicial3, best_first_graph_search, manhattan)

Solución: ['Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco aba

In [178]:
%%timeit
best_first_graph_search(p8e3,manhattan).solution()

36 ms ± 5.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


##### E4

In [199]:
resuelve_ocho_puzzle(estado_inicial4, best_first_graph_search, manhattan)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover

In [200]:
%%timeit
best_first_graph_search(p8e4,manhattan).solution()

79.1 ms ± 2.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### A* linear

##### E1

In [129]:
resuelve_ocho_puzzle(estado_inicial,astar_search,linear)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 17. Nodos analizados: 873


In [130]:
%%timeit
astar_search(p8e1, linear).solution()

180 ms ± 3.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


##### E2

In [165]:
resuelve_ocho_puzzle(estado_inicial2,astar_search,linear)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 11. Nodos analizados: 77


In [157]:
%%timeit
astar_search(p8e2, linear).solution()

2.95 ms ± 215 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### E3

In [180]:
resuelve_ocho_puzzle(estado_inicial3,astar_search,linear)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 20. Nodos analizados: 3377


In [181]:
%%timeit
astar_search(p8e3, linear).solution()

2.93 s ± 216 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### E4

In [206]:
resuelve_ocho_puzzle(estado_inicial4,astar_search,linear)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 17. Nodos analizados: 873


In [207]:
%%timeit
astar_search(p8e4, linear).solution()

226 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### A* manhattan

##### E1

In [131]:
resuelve_ocho_puzzle(estado_inicial,astar_search,manhattan)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 17. Nodos analizados: 393


In [132]:
%%timeit
astar_search(p8e1, manhattan).solution()

44.1 ms ± 3.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


##### E2

In [166]:
resuelve_ocho_puzzle(estado_inicial2,astar_search,manhattan)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 11. Nodos analizados: 42


In [160]:
%%timeit
astar_search(p8e2, manhattan).solution()

1.5 ms ± 72.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


##### E3

In [182]:
resuelve_ocho_puzzle(estado_inicial3,astar_search,manhattan)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 20. Nodos analizados: 1288


In [183]:
%%timeit
astar_search(p8e3, manhattan).solution()

442 ms ± 45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### E4

In [209]:
resuelve_ocho_puzzle(estado_inicial4,astar_search,manhattan)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 17. Nodos analizados: 393


In [210]:
%%timeit
astar_search(p8e4, manhattan).solution()

51 ms ± 3.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



### Ejercicio 6:  En el ejercicio anterior hemos hecho pruebas con distintos estados iniciales. Explica cómo afecta a la resolución del problema si lo que cambiamos es el estado objetivo.
**En vez de `goal = (1, 2, 3, 4, 5, 6, 7, 8, 0)` queremos usar `goal2 = (1, 2, 3, 4, 0, 5, 6, 7, 8)`.
No es necesario repetir los experimentos pero sí justificar convenientemente la respuesta dada.**

Al cambiar el estado objetivo es posible que algunos de los estados iniciales anteriores ya no tengan solución porque la paridad del número de inversiones totales puede cambiar, como sería el caso de usar `goal2` en vez de `goal`.

### Ejercicio 7: (opcional) Definir nuevas heurísticas (más informadas) y completa una nueva fila de la tabla anterior para ver cómo afecta al número de nodos generados por los algoritmos

In [41]:
# Proponemmos como ejemplo la heurística h3 que se obtiene sumando a la heurística
# manhattan una componente que cuantifica la "secuencialidad" en las
# casillas de un tablero, al recorrerlo en el sentido de las aguas del reloj
# ¿Es h3 admisble? Comprobar cómo se comporta esta heurística cuando se usa en
# A*, con cada uno de los estados anteriores. Comentar los resultados.

# Puedes proponer otra heuristica siempre que sea más informada que todas las heurísticas ya utilizadas.

def sucesor(i): ##Agujas del reloj
    tupla = (1, 2, 5, 0, 4, 8, 3, 6, 7)
    return tupla.index(i)

def secuencialidad(estado, i):
    valor = estado[i]
    
    ##Parte central del reloj
    if i == 4:
        return 1;
    else:
        siguiente = sucesor(i)
        
        numeroSig = valor + 1 
        if numeroSig>8:
                numeroSig = 1
                
        if numeroSig == siguiente:
            return 0
        else:
            return 2
        
def sumaSecuencialidad(node):
    sol = 0
    estado =node.state
    for i in range(8):
        if i!=0 :
            sol += secuencialidad (estado, i)
    return sol
    
def h3(node):
    return manhattan(node) + sumaSecuencialidad(node)

In [30]:
estado_inicial1 = (2,1,3,4,8,6,7,0,5)
p8h31 = Ocho_Puzzle(estado_inicial1)

In [31]:
resuelve_ocho_puzzle(estado_inicial1,astar_search,h3)

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 17. Nodos analizados: 465


In [32]:
estado_inicial2 = (1,0,3,4,8,6,7,2,5)
p8h33 = Ocho_Puzzle(estado_inicial2)

In [33]:
resuelve_ocho_puzzle(estado_inicial2,astar_search,h3) 

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 11. Nodos analizados: 65


In [34]:
estado_inicial3 = (4,5,6,1,0,3,7,8,2)
p8h33= Ocho_Puzzle(estado_inicial3)

In [35]:
resuelve_ocho_puzzle(estado_inicial3,astar_search,h3) 

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 20. Nodos analizados: 852


In [36]:
estado_inicial4 = (2,1,3,4,8,6,7,0,5)
p8h34 = Ocho_Puzzle(estado_inicial4)

In [37]:
resuelve_ocho_puzzle(estado_inicial4,astar_search,h3) 

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 17. Nodos analizados: 465


Esta heurística es más informada ya que no solamente tiene en cuenta las distancias manhattan sino que también considera las posiciones relativas con respecto a las fichas que deben ser cercanas en el sentido de las agujas del reloj. En algunos casos esta nueva heurística consigue ser mejor que `manhattan`.

### Ejercicio 8. Puzle de 15.
En 1878, Sam Loyd daba un premio de 1.000 dolares a quien fuera capaz de resolver su famoso 15 Puzzle (n=16). Se trataba de un puzzle deslizante de 16 piezas que *no tenía solución* debido que se necesitaba un número impar de movimientos y, como hemos visto, sólo un número de movimientos par tiene solución. Muchas personas trataron de resolverlo. 
El número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución. 
En el caso del siguiente estado inicial que sí tiene solución:  (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) indica los nodos generados por A* con alguna de las heurísticas y comparalo con el puzle de 8.


In [44]:
class Quince_Puzzle(Problem):
    """Problema a del 15-puzzle.  Los estados serán tuplas de 16 elementos,
    permutaciones de los números del 0 al 15 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2,3):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (12,13,14,15):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,4,8,12):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (3,7,11,15):
            accs.append("Mover hueco derecha")
        
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-4]
            l[pos_hueco - 4] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco + 1]
            l[pos_hueco + 1] = 0
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco - 1]
            l[pos_hueco - 1] = 0
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco + 4]
            l[pos_hueco + 4] = 0
        
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1

In [45]:
# Heurísticas para el puzzle de 15

def linear15(node):
    goal = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
    count = 0
    for x in node.state:
        if x != goal[x]:
            count += 1
    return count

def coords4x4(number):
    return (number/4, number%4)

def manhattan15(node):
    goal = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
    state = node.state
    mhd = 0
    for x in state:
        if x != 0:
            x_coords = coords4x4(x)
            g_coords = coords4x4(goal[x])
            mhd += abs(x_coords[0] - g_coords[0])
            mhd += abs(x_coords[1] - g_coords[1])
        
    return mhd

In [46]:
def resuelve_quince_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del quince
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       """

    p15p=Problema_con_Analizados(Quince_Puzzle(estado_inicial))
    if p15p.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p15p,h).solution()
        else: 
            sol= algoritmo(p15p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p15p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


#### Resolución del problema 15 puzle con A*

In [47]:
estado_inicial = (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) 
p15p=Problema_con_Analizados(Quince_Puzzle(estado_inicial))

##### Heurística `linear15`

In [48]:
resuelve_quince_puzzle(estado_inicial,astar_search,linear15)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear15
Longitud de la solución: 4. Nodos analizados: 34


In [49]:
%%timeit
astar_search(p15p, linear15).solution()

1.25 ms ± 29.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


##### Heurística `manhattan15`

In [50]:
resuelve_quince_puzzle(estado_inicial,astar_search,manhattan15)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan15
Longitud de la solución: 4. Nodos analizados: 34


In [51]:
%%timeit
astar_search(p15p, manhattan15).solution()

2.02 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Conclusiones
Vemos que con ambas heurísticas se analizan 34 nodos antes de llegar a la solución, la cual es de solo 4 movimientos. Funcionan con una velocidad similar a la de sus contrapartidas en el puzle de 8, salvo por el hecho de que tienen que analizar más datos al ser el tablero del puzle de 15 más grande que el de 8.

Siendo el estado inicial del que partimos muy similar al estado final `goal` la búsqueda de la solución no tarda mucho, pero teniendo el puzle de 15 muchos más estados posibles que el de 8 este tiempo podría haberse disparado con otro estado inicial.